# Dynamically pin and unpin from providers
## Benefit 
* Data Scientists can dynamically change pinning providers
* Data Scientists don't rely on one IPFS gateway, but can use many


In [4]:
help(pinata.upload_file)

Help on method upload_file in module storage.pinataV1:

upload_file(path, cred, fn, pinataMetadata={}) method of storage.pinataV1.PinataV1 instance



In [4]:
data = {"ipfsPinHash": "QmNN12xSayeJ7m3yz5bK4bVb4rJ4cr8WS6EcNeMKDcfZsE",
        "name": "compound.csv",
        "keyvalues": {
            "DAO":"DeepFi",
            "Data":"Compound"
        }
         }

pinata.upload_file("dataset/sample/compound.csv",pinata_creds,"compound.csv",pinataMetadata=data)

({'IpfsHash': 'QmWAQjxm6CKaAjHjwPSqWmN8RMecuHS1bxEufqCNVSq96e',
  'PinSize': 5985301,
  'Timestamp': '2022-03-29T16:06:53.647Z'},
 200)

200

## Importing libraries

In [1]:
#main protocol
from storage.ipfs import IPFS

#pinning providers
from storage.nftstorage import NFTStorage
from storage.pinataV1 import PinataV1


#helpers
from helpers.helper import read_file
import pandas as pd

In [2]:
#Load in credentials - Not an optimal solution and can investigate a better solution with something like Lit Protocol

nft_storage = NFTStorage()
nft_storage_creds = nft_storage.get_creds()
ipfs = IPFS()
pinata = PinataV1()
pinata_creds = pinata.get_creds()

In [4]:
#Upload to nft storage, this is called pinning or simplicity, NFTstorage actually facilitates the pinning deals under the hood inteh deals section
#the response returns a cid and what filecoin deals have been made to pin your file.

response, status_code = nft_storage.upload_file("dataset/sample/ens_airdrop.json",nft_storage_creds)

pd.DataFrame(response)

,ok,value
cid,True,bafybeiek3lyobn6lgt46bj5pf4cyd7qcj7o4umkeyg7rv...
created,True,2022-03-09T02:21:11.461+00:00
deals,True,"[{'status': 'active', 'lastChanged': '2022-03-..."
files,True,[]
pin,True,{'cid': 'bafybeiek3lyobn6lgt46bj5pf4cyd7qcj7o4...
scope,True,audiocentive
size,True,42725339
type,True,form-data


In [5]:
#Pin to pinata for redundency
pinata.pin(response["value"]["cid"],pinata_creds,fn="ens_airdrop.json")

({'id': 'ea70bcad-f7d7-4c96-aa92-53a3ecadc1d2',
  'ipfsHash': 'bafybeiek3lyobn6lgt46bj5pf4cyd7qcj7o4umkeyg7rvwkpjfr6tsj3ye',
  'status': 'prechecking',
  'name': 'ens_airdrop.json'},
 200)

In [6]:
#Check the status of the pinning job, pin policy will show what regions the file is available in
params = {"hashContains":"DESC","status":None,"prechecking":None,
          "searching":None,"retrieving":None,
          "expired":None,"over_free_limit":None,
          "over_max_size":None,"invalid_object":None,
          "bad_host_node":None,"ipfs_pin_hash":response["value"]["cid"],
          "limit":None,"offset":None,"metadata[name]":None,
          "metadatakeyvalues":"keyvalues"
         }

pinata.get_pinned_jobs(pinata_creds,params)

NameError: name 'response' is not defined

In [7]:
"""
Query Parameters = params

hashContains: (string) - Filter on alphanumeric characters inside of pin hashes. Hashes which do not include the characters passed in will not be returned.

pinStart: (must be in ISO_8601 format) - Exclude pin records that were pinned before the passed in "pinStart" datetime.

pinEnd: (must be in ISO_8601 format) - Exclude pin records that were pinned after the passed in "pinEnd" datetime.

unpinStart: (must be in ISO_8601 format) - Exclude pin records that were unpinned before the passed in "unpinStart" datetime.

unpinEnd: (must be in ISO_8601 format) - Exclude pin records that were unpinned after the passed in "unpinEnd" datetime.

pinSizeMin: (integer) - The minimum byte size that pin record you're looking for can have

pinSizeMax: (integer) - The maximum byte size that pin record you're looking for can have

status: (string) -
    * Pass in "all" for both pinned and unpinned records
    * Pass in "pinned" for just pinned records (hashes that are currently pinned)
    * Pass in "unpinned" for just unpinned records (previous hashes that are no longer being pinned on pinata)

pageLimit: (integer) - This sets the amount of records that will be returned per API response. (Max 1000)

pageOffset: (integer) - This tells the API how far to offset the record responses. For example, 
if there's 30 records that match your query, and you passed in a pageLimit of 10, providing a pageOffset of 10 would return records 11-20.
"""

#Get all pinned files to unpin and then pin in NFT Storage
#If pageLimit == None then defaults to most recent 10

params = {"hasContains":response["value"]["cid"],"pinStart":None,
          "pinEnd":None,"unpinStart":None,
          "unpinEnd":None,"pinSizeMin":None,
          "pinSizeMax":None,"status":None,
          "pageLimit":30,"pageOffset":None,
         }

pf,status = pinata.get_pinned_files(pinata_creds,params)
df_pinata = pd.DataFrame(pf["rows"])

df_pinata.drop_duplicates(["ipfs_pin_hash"])

,id,ipfs_pin_hash,size,user_id,date_pinned,date_unpinned,metadata,regions
0,7a56f757-79f3-46ea-ad55-f9c8a5763143,bafybeiek3lyobn6lgt46bj5pf4cyd7qcj7o4umkeyg7rv...,42725339,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-03-29T03:13:36.976Z,None,"{'name': None, 'keyvalues': {}}","[{'regionId': 'NYC1', 'currentReplicationCount..."
1,c8323cfe-63fb-4877-8777-641b24316fad,QmQwwMbykhX3wyCdC5yBRXV4JQRe5FFLiyynaUiLbmvLzn,10013,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-03-29T03:13:35.557Z,2022-03-29T03:13:36.395Z,"{'name': None, 'keyvalues': {}}","[{'regionId': 'NYC1', 'currentReplicationCount..."
2,bbefeb61-64f9-4901-ae3c-5c73c04ccba6,QmZ8bNMNXodiWRJDvDHNCAE1EsnbaiTZaQJafm2pSTYBFh,9887,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-03-29T03:13:35.295Z,None,"{'name': None, 'keyvalues': {}}","[{'regionId': 'NYC1', 'currentReplicationCount..."
3,a7bc087e-6748-4be5-aae6-e420aabeefe3,bafybeif6gdxdpleultgqobk65dhy6gvnozu27a3lbm4xw...,6455453,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-03-29T03:07:22.902Z,None,"{'name': 'aave.csv', 'keyvalues': {'0': 'newKe...","[{'regionId': 'NYC1', 'currentReplicationCount..."
5,fcfc03c5-a24e-46fe-974d-81c50530a06b,QmQxh7mQhpuCCSt7ryU2MCqc4zdoxVQfSJF4Mun2AUTVqd,2130,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-03-28T21:48:34.004Z,2022-03-28T22:26:02.279Z,"{'name': None, 'keyvalues': {}}","[{'regionId': 'NYC1', 'currentReplicationCount..."
14,c3106d6e-d8f6-4d4f-8186-93f89d8e8f15,bafkreiesap4b4a4xgvfxhsz6hdza7atezclk4op3iz5lu...,23597,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-03-24T16:56:19.857Z,None,{'name': 'Popcorn Limited - Dune Dashboards - ...,"[{'regionId': 'NYC1', 'currentReplicationCount..."
15,838cd171-f6c6-40a7-beec-899ca8acb66d,bafkreief472lhr6b54baq4oansiwor4ed2te2xztwgjqe...,24858,d28f20cf-dfe0-4c8f-9272-26b9c2d8a3cd,2022-03-24T16:54:09.843Z,None,{'name': 'Popcorn Limited - Dune Dashboards - ...,"[{'regionId': 'NYC1', 'currentReplicationCount..."


In [8]:
#Make a CID dictionary to pin these files to NFT Storage

df_pinata["filename"]=df_pinata["metadata"].apply(lambda x: x["name"])
file_dict = df_pinata[["filename","ipfs_pin_hash"]].drop_duplicates(subset="filename").set_index("filename")

file_dict = file_dict.drop(index=["Popcorn Limited - Dune Dashboards - 03242022.pdf","Popcorn Limited - Dune Dashboards - 03182022.pdf"])

file_dict

,ipfs_pin_hash
filename,
NaN,bafybeiek3lyobn6lgt46bj5pf4cyd7qcj7o4umkeyg7rv...
aave.csv,bafybeif6gdxdpleultgqobk65dhy6gvnozu27a3lbm4xw...
ens_airdrop.json,bafybeiek3lyobn6lgt46bj5pf4cyd7qcj7o4umkeyg7rv...
All_Gateway_Properties.csv,QmQwwMbykhX3wyCdC5yBRXV4JQRe5FFLiyynaUiLbmvLzn
dataset,QmQwwMbykhX3wyCdC5yBRXV4JQRe5FFLiyynaUiLbmvLzn
compound.csv,QmQwwMbykhX3wyCdC5yBRXV4JQRe5FFLiyynaUiLbmvLzn
yes,QmZ8bNMNXodiWRJDvDHNCAE1EsnbaiTZaQJafm2pSTYBFh


In [9]:
#NFT Storage Files

response = nft_storage.get_all_files(nft_storage_creds)

nft_storage_files = pd.DataFrame(response.json()["value"])

nft_storage_files

,cid,created,type,scope,files,size,pin,deals
0,bafkreiesap4b4a4xgvfxhsz6hdza7atezclk4op3iz5lu...,2022-03-24T16:55:42.506+00:00,form-data,audiocentive,[],23597,{'cid': 'bafkreiesap4b4a4xgvfxhsz6hdza7atezclk...,"[{'status': 'active', 'lastChanged': '2022-03-..."
1,bafkreief472lhr6b54baq4oansiwor4ed2te2xztwgjqe...,2022-03-24T16:51:43.101+00:00,form-data,audiocentive,[],24858,{'cid': 'bafkreief472lhr6b54baq4oansiwor4ed2te...,"[{'status': 'active', 'lastChanged': '2022-03-..."
2,bafybeif6gdxdpleultgqobk65dhy6gvnozu27a3lbm4xw...,2022-03-22T17:04:21.124+00:00,form-data,audiocentive,[],6455453,{'cid': 'bafybeif6gdxdpleultgqobk65dhy6gvnozu2...,"[{'status': 'active', 'lastChanged': '2022-03-..."
3,bafkreifhvosw3d4agsnduodmyupqdngsjdddobi5altxt...,2022-03-21T15:57:38.91+00:00,application/car,session,[],2119,{'cid': 'bafkreifhvosw3d4agsnduodmyupqdngsjddd...,"[{'status': 'active', 'lastChanged': '2022-03-..."
4,bafkreie2muwin7je3fc5uaepmc6jwt6mv6mfxiiwbm2yn...,2022-03-20T23:33:25.689+00:00,form-data,audiocentive,[],895,{'cid': 'bafkreie2muwin7je3fc5uaepmc6jwt6mv6mf...,"[{'status': 'active', 'lastChanged': '2022-03-..."
5,bafkreicwnshhm5jg6uvf6lvfcebohrijpyb6srz2vu2mj...,2022-03-09T06:19:39.874+00:00,form-data,audiocentive,[],10060,{'cid': 'bafkreicwnshhm5jg6uvf6lvfcebohrijpyb6...,"[{'status': 'active', 'lastChanged': '2022-03-..."
6,bafybeibjwiqi4c7yebpobalni3eijfgkgwmidbuyacyz6...,2022-03-09T04:10:04.14+00:00,image/jpeg,audiocentive,[],5245419,{'cid': 'bafybeibjwiqi4c7yebpobalni3eijfgkgwmi...,"[{'status': 'active', 'lastChanged': '2022-03-..."
7,bafybeif36uqfqj5on23qfdl3nglc7ncgcbnz4ixpleoms...,2022-03-09T04:03:21.867+00:00,form-data,audiocentive,[],2086898,{'cid': 'bafybeif36uqfqj5on23qfdl3nglc7ncgcbnz...,"[{'status': 'active', 'lastChanged': '2022-03-..."
8,bafybeihngw4os7fg6xzthb7cpydfiavpufkg746khabkc...,2022-03-09T03:16:57.338+00:00,directory,audiocentive,"[{'name': 'myfile', 'type': ''}]",42725396,{'cid': 'bafybeihngw4os7fg6xzthb7cpydfiavpufkg...,"[{'status': 'active', 'lastChanged': '2022-03-..."
9,bafybeifxdk6cm6xecwyq7euxojpiuao66ob3nkbjjpxhh...,2022-03-09T02:31:11.692+00:00,form-data,audiocentive,[],42725615,{'cid': 'bafybeifxdk6cm6xecwyq7euxojpiuao66ob3...,"[{'status': 'active', 'lastChanged': '2022-03-..."


# Compare Pinata Files to NFT Storage to upload the outstanding files

In [10]:
pinata_pinned = file_dict["ipfs_pin_hash"].to_list()

nft_storage_pinned = nft_storage_files["cid"].to_list()

outstanding_hashs = set(pinata_pinned).difference(nft_storage_pinned)
outstanding_hashs

{'QmQwwMbykhX3wyCdC5yBRXV4JQRe5FFLiyynaUiLbmvLzn',
 'QmZ8bNMNXodiWRJDvDHNCAE1EsnbaiTZaQJafm2pSTYBFh',
 'bafybeiek3lyobn6lgt46bj5pf4cyd7qcj7o4umkeyg7rvwkpjfr6tsj3ye'}

In [11]:
#Pin oustanding hashes to pinata
for cid in outstanding_hashs:
    
    pinata.pin(cid,pinata_creds)

In [12]:
#Pin oustanding hashes to pinata

for cid in outstanding_hashs:
    pinata.unpin(cid,pinata_creds)
    nft_storage.unpin(cid,nft_storage_creds)

# As you can see all the CIDs with a Q were removed. Q is V1 and b is V2

In [13]:
response = nft_storage.get_all_files(nft_storage_creds)

nft_storage_files = pd.DataFrame(response.json()["value"])

nft_storage_files

,cid,created,type,scope,files,size,pin,deals
0,bafkreiesap4b4a4xgvfxhsz6hdza7atezclk4op3iz5lu...,2022-03-24T16:55:42.506+00:00,form-data,audiocentive,[],23597,{'cid': 'bafkreiesap4b4a4xgvfxhsz6hdza7atezclk...,"[{'status': 'active', 'lastChanged': '2022-03-..."
1,bafkreief472lhr6b54baq4oansiwor4ed2te2xztwgjqe...,2022-03-24T16:51:43.101+00:00,form-data,audiocentive,[],24858,{'cid': 'bafkreief472lhr6b54baq4oansiwor4ed2te...,"[{'status': 'active', 'lastChanged': '2022-03-..."
2,bafybeif6gdxdpleultgqobk65dhy6gvnozu27a3lbm4xw...,2022-03-22T17:04:21.124+00:00,form-data,audiocentive,[],6455453,{'cid': 'bafybeif6gdxdpleultgqobk65dhy6gvnozu2...,"[{'status': 'active', 'lastChanged': '2022-03-..."
3,bafkreifhvosw3d4agsnduodmyupqdngsjdddobi5altxt...,2022-03-21T15:57:38.91+00:00,application/car,session,[],2119,{'cid': 'bafkreifhvosw3d4agsnduodmyupqdngsjddd...,"[{'status': 'active', 'lastChanged': '2022-03-..."
4,bafkreie2muwin7je3fc5uaepmc6jwt6mv6mfxiiwbm2yn...,2022-03-20T23:33:25.689+00:00,form-data,audiocentive,[],895,{'cid': 'bafkreie2muwin7je3fc5uaepmc6jwt6mv6mf...,"[{'status': 'active', 'lastChanged': '2022-03-..."
5,bafkreicwnshhm5jg6uvf6lvfcebohrijpyb6srz2vu2mj...,2022-03-09T06:19:39.874+00:00,form-data,audiocentive,[],10060,{'cid': 'bafkreicwnshhm5jg6uvf6lvfcebohrijpyb6...,"[{'status': 'active', 'lastChanged': '2022-03-..."
6,bafybeibjwiqi4c7yebpobalni3eijfgkgwmidbuyacyz6...,2022-03-09T04:10:04.14+00:00,image/jpeg,audiocentive,[],5245419,{'cid': 'bafybeibjwiqi4c7yebpobalni3eijfgkgwmi...,"[{'status': 'active', 'lastChanged': '2022-03-..."
7,bafybeif36uqfqj5on23qfdl3nglc7ncgcbnz4ixpleoms...,2022-03-09T04:03:21.867+00:00,form-data,audiocentive,[],2086898,{'cid': 'bafybeif36uqfqj5on23qfdl3nglc7ncgcbnz...,"[{'status': 'active', 'lastChanged': '2022-03-..."
8,bafybeihngw4os7fg6xzthb7cpydfiavpufkg746khabkc...,2022-03-09T03:16:57.338+00:00,directory,audiocentive,"[{'name': 'myfile', 'type': ''}]",42725396,{'cid': 'bafybeihngw4os7fg6xzthb7cpydfiavpufkg...,"[{'status': 'active', 'lastChanged': '2022-03-..."
9,bafybeifxdk6cm6xecwyq7euxojpiuao66ob3nkbjjpxhh...,2022-03-09T02:31:11.692+00:00,form-data,audiocentive,[],42725615,{'cid': 'bafybeifxdk6cm6xecwyq7euxojpiuao66ob3...,"[{'status': 'active', 'lastChanged': '2022-03-..."


In [14]:
nft_storage_files.to_dict()["cid"][6]

'bafybeibjwiqi4c7yebpobalni3eijfgkgwmidbuyacyz6hawfkrporatpi'

In [15]:
from storage.ipfs import IPFS

ipfs = IPFS()

In [6]:
response,log = ipfs.get_file("QmWAQjxm6CKaAjHjwPSqWmN8RMecuHS1bxEufqCNVSq96e",local_node=False)

Retrieved file hash QmWAQjxm6CKaAjHjwPSqWmN8RMecuHS1bxEufqCNVSq96e from https://cf-ipfs.com Response 200


In [7]:
read_file(response.content)

,Timestamp,Token,Borrowing Rate,Deposit Rate,Borrow Volume,Supply Volume
0,1609471800,DAI,0.073195,0.050982,1.069964e+09,6.196481e+10
1,1609471800,USDC,0.087046,0.066993,7.285430e+08,4.063042e+10
2,1609471800,USDT,0.099588,0.077548,6.430536e+07,3.696225e+09
3,1609471800,ETH,0.022952,0.000489,3.055365e+04,5.663257e+07
4,1609473600,DAI,0.073101,0.050912,1.069961e+09,6.197050e+10
...,...,...,...,...,...,...
66789,1639539000,ETH,0.027215,0.000761,5.316478e+04,7.481990e+07
66790,1639540800,ETH,0.027214,0.000761,5.316480e+04,7.482188e+07
66791,1639542600,ETH,0.027214,0.000761,5.316488e+04,7.482482e+07
66792,1639544400,ETH,0.027226,0.000764,5.316501e+04,7.461519e+07


In [ ]:
data = {"ipfsPinHash": "QmWAQjxm6CKaAjHjwPSqWmN8RMecuHS1bxEufqCNVSq96e",
        "name": "compound.csv",
        "keyvalues": {
            "DAO":"DeepFi",
            "Data":"Compound"
        }
         }

pinata.edit_hash("QmWAQjxm6CKaAjHjwPSqWmN8RMecuHS1bxEufqCNVSq96e",pinata_creds,"compound.csv",pinataMetaData=data)